In [2]:
!pip install pandas numpy scikit-learn tensorflow imblearn gradio matplotlib seaborn
!pip install gradio
!pip install scikeras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 119.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.6 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, roc_auc_score
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Add, Softmax
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Reshape, MultiHeadAttention, LayerNormalization
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow import keras
import gradio as gr
from sklearn.preprocessing import PolynomialFeatures
from sklearn.utils import compute_class_weight
from google.colab import drive
from google.colab import files

tf.random.set_seed(42)
np.random.seed(42)

drive.mount('/content/drive')

drive_model_dir = '/content/drive/MyDrive/ML_Churn_Models'
os.makedirs(drive_model_dir, exist_ok=True)

models = {
    'simple_mlp': ['Adam', 'RMSprop', 'SGD'],
    'tabnet': ['Adam', 'RMSprop', 'SGD'],
    'transformer': ['Adam', 'RMSprop', 'SGD'],
    'residual_mlp': ['Adam', 'RMSprop', 'SGD'],
    'cnn_mlp': ['Adam', 'RMSprop', 'SGD']
}

data = pd.read_csv('credit_card_churn.csv')

data = data.drop(columns=['CLIENTNUM',
                         'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1',
                         'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'])

X = data.drop(columns=['Attrition_Flag'])
y = data['Attrition_Flag'].apply(lambda x: 1 if x == 'Attrited Customer' else 0)

categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)
    ])


Mounted at /content/drive


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_poly, y_train)

input_shape = X_train_smote.shape[1]

class_weights = compute_class_weight('balanced', classes=np.unique(y_train_smote), y=y_train_smote)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

optimizers = {
    'Adam': Adam(learning_rate=0.001),
    'RMSprop': RMSprop(learning_rate=0.001),
    'SGD': SGD(learning_rate=0.01, momentum=0.9)
}

In [5]:
def build_simple_mlp(optimizer=Adam(learning_rate=0.001)):
    model = Sequential([
        Input(shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model


def build_tabnet_inspired_model(optimizer=Adam(learning_rate=0.001)):
    inputs = Input(shape=(input_shape,))
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    shared_features = x

    outputs = []
    mask = None
    attention_dim = 64
    for step in range(3):
        if mask is None:
            attn = Dense(attention_dim, activation=None)(shared_features)
        else:
            attn = Dense(attention_dim, activation=None)(mask)
        attn = Softmax()(attn)
        masked_features = shared_features * attn

        x = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(masked_features)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)
        outputs.append(x)
        mask = x

    outputs = []
    initial_mask = Dense(64, activation='relu')(shared_features)
    mask = Softmax()(initial_mask)

    for step in range(3):
        masked_features = shared_features * mask

        x = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(masked_features)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)
        outputs.append(x)


        if step < 2:
             next_mask_input = Dense(64, activation='relu')(x)
             mask = Softmax()(next_mask_input)


    x = tf.keras.layers.Concatenate()(outputs)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_transformer_model(optimizer=Adam(learning_rate=0.001)):
    inputs = Input(shape=(input_shape,))
    x = Dense(64, activation='relu')(inputs)
    x = Reshape((1, 64))(x)
    attention = MultiHeadAttention(num_heads=4, key_dim=16)(x, x)
    x = LayerNormalization()(attention)
    x = Flatten()(x)
    x = Dense(32, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_residual_mlp(optimizer=Adam(learning_rate=0.001)):
    inputs = Input(shape=(input_shape,))
    x = Dense(64, activation='relu')(inputs)
    x = Dense(64, activation='relu')(x)
    residual = x
    x = Dense(64, activation='relu')(x)
    x = Add()([x, residual])
    x = Dense(32, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def build_cnn_inspired_mlp(optimizer=Adam(learning_rate=0.001)):
    model = Sequential([
        Input(shape=(input_shape,)),
        Reshape((input_shape, 1)),
        Conv1D(32, kernel_size=3, activation='relu', padding='same'),
        MaxPooling1D(pool_size=2),
        Conv1D(16, kernel_size=3, activation='relu', padding='same'),
        Flatten(),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

results = {}
best_models = {}

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [6]:
print("\nTraining Simple MLP with Adam...")
simple_mlp_adam = build_simple_mlp(optimizers['Adam'])
simple_mlp_adam.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
simple_mlp_adam.save('simple_mlp_adam.keras')
print("Saved Simple MLP with Adam weights")
y_pred_proba = simple_mlp_adam.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['Simple MLP_Adam'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['Simple MLP_Adam'] = simple_mlp_adam
print(f"Simple MLP_Adam - Accuracy: {results['Simple MLP_Adam']['accuracy']:.4f}, ROC-AUC: {results['Simple MLP_Adam']['roc_auc']:.4f}")
print(f"Simple MLP_Adam - precision: {results['Simple MLP_Adam']['precision']:.4f}")
print(f"Simple MLP_Adam - recall: {results['Simple MLP_Adam']['recall']:.4f}")
print(f"Simple MLP_Adam - f1_score: {results['Simple MLP_Adam']['f1_score']:.4f}")

print("\nTraining Simple MLP with RMSprop...")
paziente = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
simple_mlp_rmsprop = build_simple_mlp(optimizers['RMSprop'])
simple_mlp_rmsprop.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
simple_mlp_rmsprop.save('simple_mlp_rmsprop.keras')
print("Saved Simple MLP with RMSProp weights")
y_pred_proba = simple_mlp_rmsprop.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['Simple MLP_RMSprop'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['Simple MLP_RMSprop'] = simple_mlp_rmsprop
print(f"Simple MLP_RMSprop - Accuracy: {results['Simple MLP_RMSprop']['accuracy']:.4f}, ROC-AUC: {results['Simple MLP_RMSprop']['roc_auc']:.4f}")
print(f"Simple MLP_RMSprop - precision: {results['Simple MLP_RMSprop']['precision']:.4f}")
print(f"Simple MLP_RMSprop - recall: {results['Simple MLP_RMSprop']['recall']:.4f}")
print(f"Simple MLP_RMSprop - f1_score: {results['Simple MLP_RMSprop']['f1_score']:.4f}")

print("\nTraining Simple MLP with SGD...")
simple_mlp_sgd = build_simple_mlp(optimizers['SGD'])
simple_mlp_sgd.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
simple_mlp_sgd.save('simple_mlp_sgd.keras')
print("Saved Simple MLP with SGD weights")
y_pred_proba = simple_mlp_sgd.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['Simple MLP_SGD'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['Simple MLP_SGD'] = simple_mlp_sgd
print(f"Simple MLP_SGD - Accuracy: {results['Simple MLP_SGD']['accuracy']:.4f}, ROC-AUC: {results['Simple MLP_SGD']['roc_auc']:.4f}")
print(f"Simple MLP_SGD - precision: {results['Simple MLP_SGD']['precision']:.4f}")
print(f"Simple MLP_SGD - recall: {results['Simple MLP_SGD']['recall']:.4f}")
print(f"Simple MLP_SGD - f1_score: {results['Simple MLP_SGD']['f1_score']:.4f}")



Training Simple MLP with Adam...
Saved Simple MLP with Adam weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
Simple MLP_Adam - Accuracy: 0.9181, ROC-AUC: 0.9496
Simple MLP_Adam - precision: 0.7657
Simple MLP_Adam - recall: 0.7095
Simple MLP_Adam - f1_score: 0.7365

Training Simple MLP with RMSprop...
Saved Simple MLP with RMSProp weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Simple MLP_RMSprop - Accuracy: 0.9191, ROC-AUC: 0.9523
Simple MLP_RMSprop - precision: 0.7508
Simple MLP_RMSprop - recall: 0.7462
Simple MLP_RMSprop - f1_score: 0.7485

Training Simple MLP with SGD...
Saved Simple MLP with SGD weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
Simple MLP_SGD - Accuracy: 0.9210, ROC-AUC: 0.9523
Simple MLP_SGD - precision: 0.7685
Simple MLP_SGD - recall: 0.7309
Simple MLP_SGD - f1_score: 0.7492


In [7]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_smote), y=y_train_smote)
class_weight_dict = dict(enumerate(class_weights))
# print(f"\nCalculated Class Weights: {class_weight_dict}")

print("\nTraining TabNet-inspired Model with Adam...")
tabnet_adam = build_tabnet_inspired_model(optimizers['Adam'])
tabnet_adam.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
tabnet_adam.save('tabnet_adam.keras')
print("Saved TabNet with Adam weights")
y_pred_proba = tabnet_adam.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['TabNet-inspired Model_Adam'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['TabNet-inspired Model_Adam'] = tabnet_adam
print(f"TabNet-inspired Model_Adam - Accuracy: {results['TabNet-inspired Model_Adam']['accuracy']:.4f}, ROC-AUC: {results['TabNet-inspired Model_Adam']['roc_auc']:.4f}")
print(f"TabNet-inspired Model_Adam - precision: {results['TabNet-inspired Model_Adam']['precision']:.4f}")
print(f"TabNet-inspired Model_Adam - recall: {results['TabNet-inspired Model_Adam']['recall']:.4f}")
print(f"TabNet-inspired Model_Adam - f1_score: {results['TabNet-inspired Model_Adam']['f1_score']:.4f}")

print("\nTraining TabNet-inspired Model with RMSprop...")
tabnet_rmsprop = build_tabnet_inspired_model(optimizers['RMSprop'])
tabnet_rmsprop.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
tabnet_rmsprop.save('tabnet_rmsprop.keras')
print("Saved TabNet with RMSProp weights")
y_pred_proba = tabnet_rmsprop.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['TabNet-inspired Model_RMSprop'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['TabNet-inspired Model_RMSprop'] = tabnet_rmsprop
print(f"TabNet-inspired Model_RMSprop - Accuracy: {results['TabNet-inspired Model_RMSprop']['accuracy']:.4f}, ROC-AUC: {results['TabNet-inspired Model_RMSprop']['roc_auc']:.4f}")
print(f"TabNet-inspired Model_RMSprop - precision: {results['TabNet-inspired Model_RMSprop']['precision']:.4f}")
print(f"TabNet-inspired Model_RMSprop - recall: {results['TabNet-inspired Model_RMSprop']['recall']:.4f}")
print(f"TabNet-inspired Model_RMSprop - f1_score: {results['TabNet-inspired Model_RMSprop']['f1_score']:.4f}")

print("\nTraining TabNet-inspired Model with SGD...")
tabnet_sgd = build_tabnet_inspired_model(optimizers['SGD'])
tabnet_sgd.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
tabnet_sgd.save('tabnet_sgd.keras')
print("Saved TabNet with SGD weights")
y_pred_proba = tabnet_sgd.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['TabNet-inspired Model_SGD'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['TabNet-inspired Model_SGD'] = tabnet_sgd
print(f"TabNet-inspired Model_SGD - Accuracy: {results['TabNet-inspired Model_SGD']['accuracy']:.4f}, ROC-AUC: {results['TabNet-inspired Model_SGD']['roc_auc']:.4f}")
print(f"TabNet-inspired Model_SGD - precision: {results['TabNet-inspired Model_SGD']['precision']:.4f}")
print(f"TabNet-inspired Model_SGD - recall: {results['TabNet-inspired Model_SGD']['recall']:.4f}")
print(f"TabNet-inspired Model_SGD - f1_score: {results['TabNet-inspired Model_SGD']['f1_score']:.4f}")


Training TabNet-inspired Model with Adam...
Saved TabNet with Adam weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
TabNet-inspired Model_Adam - Accuracy: 0.9200, ROC-AUC: 0.9587
TabNet-inspired Model_Adam - precision: 0.7236
TabNet-inspired Model_Adam - recall: 0.8165
TabNet-inspired Model_Adam - f1_score: 0.7672

Training TabNet-inspired Model with RMSprop...
Saved TabNet with RMSProp weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step
TabNet-inspired Model_RMSprop - Accuracy: 0.9028, ROC-AUC: 0.9564
TabNet-inspired Model_RMSprop - precision: 0.6519
TabNet-inspired Model_RMSprop - recall: 0.8532
TabNet-inspired Model_RMSprop - f1_score: 0.7391

Training TabNet-inspired Model with SGD...
Saved TabNet with SGD weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
TabNet-inspired Model_SGD - Accuracy: 0.9171, ROC-AUC: 0.9591
TabNet-inspired Model_SGD - precision: 0.7143
TabNet-inspired Model_SGD - recall: 0.8104
TabNet-inspired Model_SGD - f1_score: 0.7593


In [8]:
print("\nTraining Transformer Model with Adam...")
transformer_adam = build_transformer_model(optimizers['Adam'])
transformer_adam.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
transformer_adam.save('transformer_adam.keras')
print("Saved Transformer with Adam weights")
y_pred_proba = transformer_adam.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['Transformer Model_Adam'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['Transformer Model_Adam'] = transformer_adam
print(f"Transformer Model_Adam - Accuracy: {results['Transformer Model_Adam']['accuracy']:.4f}, ROC-AUC: {results['Transformer Model_Adam']['roc_auc']:.4f}")
print(f"Transformer Model_Adam - precision: {results['Transformer Model_Adam']['precision']:.4f}")
print(f"Transformer Model_Adam - recall: {results['Transformer Model_Adam']['recall']:.4f}")
print(f"Transformer Model_Adam - f1_score: {results['Transformer Model_Adam']['f1_score']:.4f}")

print("\nTraining Transformer Model with RMSprop...")
transformer_rmsprop = build_transformer_model(optimizers['RMSprop'])
transformer_rmsprop.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
transformer_rmsprop.save('transformer_rmsprop.keras')
print("Saved Transformer with RMSProp weights")
y_pred_proba = transformer_rmsprop.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['Transformer Model_RMSprop'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['Transformer Model_RMSprop'] = transformer_rmsprop
print(f"Transformer Model_RMSprop - Accuracy: {results['Transformer Model_RMSprop']['accuracy']:.4f}, ROC-AUC: {results['Transformer Model_RMSprop']['roc_auc']:.4f}")
print(f"Transformer Model_RMSprop - precision: {results['Transformer Model_RMSprop']['precision']:.4f}")
print(f"Transformer Model_RMSprop - recall: {results['Transformer Model_RMSprop']['recall']:.4f}")
print(f"Transformer Model_RMSprop - f1_score: {results['Transformer Model_RMSprop']['f1_score']:.4f}")

print("\nTraining Transformer Model with SGD...")
transformer_sgd = build_transformer_model(optimizers['SGD'])
transformer_sgd.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
transformer_sgd.save('transformer_sgd.keras')
print("Saved Transformer with SGD weights")
y_pred_proba = transformer_sgd.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['Transformer Model_SGD'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['Transformer Model_SGD'] = transformer_sgd
print(f"Transformer Model_SGD - Accuracy: {results['Transformer Model_SGD']['accuracy']:.4f}, ROC-AUC: {results['Transformer Model_SGD']['roc_auc']:.4f}")
print(f"Transformer Model_SGD - precision: {results['Transformer Model_SGD']['precision']:.4f}")
print(f"Transformer Model_SGD - recall: {results['Transformer Model_SGD']['recall']:.4f}")
print(f"Transformer Model_SGD - f1_score: {results['Transformer Model_SGD']['f1_score']:.4f}")


Training Transformer Model with Adam...
Saved Transformer with Adam weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Transformer Model_Adam - Accuracy: 0.9156, ROC-AUC: 0.9428
Transformer Model_Adam - precision: 0.7438
Transformer Model_Adam - recall: 0.7278
Transformer Model_Adam - f1_score: 0.7357

Training Transformer Model with RMSprop...
Saved Transformer with RMSProp weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Transformer Model_RMSprop - Accuracy: 0.9146, ROC-AUC: 0.9404
Transformer Model_RMSprop - precision: 0.7175
Transformer Model_RMSprop - recall: 0.7768
Transformer Model_RMSprop - f1_score: 0.7460

Training Transformer Model with SGD...
Saved Transformer with SGD weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
Transformer Model_SGD - Accuracy: 0.9136, ROC-AUC: 0.9483
Transformer Model_SGD - precision: 0.7568
Transformer Model_SGD - recall: 0.6850
Transformer Model_SGD - f1_score: 0.7191


In [9]:
print("\nTraining Residual MLP with Adam...")
residual_mlp_adam = build_residual_mlp(optimizers['Adam'])
residual_mlp_adam.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
residual_mlp_adam.save('residual_mlp_adam.keras')
print("Saved Residual MLP with Adam weights")
y_pred_proba = residual_mlp_adam.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['Residual MLP_Adam'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['Residual MLP_Adam'] = residual_mlp_adam
print(f"Residual MLP_Adam - Accuracy: {results['Residual MLP_Adam']['accuracy']:.4f}, ROC-AUC: {results['Residual MLP_Adam']['roc_auc']:.4f}")
print(f"Residual MLP_Adam - precision: {results['Residual MLP_Adam']['precision']:.4f}")
print(f"Residual MLP_Adam - recall: {results['Residual MLP_Adam']['recall']:.4f}")
print(f"Residual MLP_Adam - f1_score: {results['Residual MLP_Adam']['f1_score']:.4f}")

print("\nTraining Residual MLP with RMSprop...")
residual_mlp_rmsprop = build_residual_mlp(optimizers['RMSprop'])
residual_mlp_rmsprop.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
residual_mlp_rmsprop.save('residual_mlp_rmsprop.keras')
print("Saved Residual MLP with RMSProp weights")
y_pred_proba = residual_mlp_rmsprop.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['Residual MLP_RMSprop'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['Residual MLP_RMSprop'] = residual_mlp_rmsprop
print(f"Residual MLP_RMSprop - Accuracy: {results['Residual MLP_RMSprop']['accuracy']:.4f}, ROC-AUC: {results['Residual MLP_RMSprop']['roc_auc']:.4f}")
print(f"Residual MLP_RMSprop - precision: {results['Residual MLP_RMSprop']['precision']:.4f}")
print(f"Residual MLP_RMSprop - recall: {results['Residual MLP_RMSprop']['recall']:.4f}")
print(f"Residual MLP_RMSprop - f1_score: {results['Residual MLP_RMSprop']['f1_score']:.4f}")

print("\nTraining Residual MLP with SGD...")
residual_mlp_sgd = build_residual_mlp(optimizers['SGD'])
residual_mlp_sgd.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
residual_mlp_sgd.save('residual_mlp_sgd.keras')
print("Saved Residual MLP with SGD weights")
y_pred_proba = residual_mlp_sgd.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['Residual MLP_SGD'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['Residual MLP_SGD'] = residual_mlp_sgd
print(f"Residual MLP_SGD - Accuracy: {results['Residual MLP_SGD']['accuracy']:.4f}, ROC-AUC: {results['Residual MLP_SGD']['roc_auc']:.4f}")
print(f"Residual MLP_SGD - precision: {results['Residual MLP_SGD']['precision']:.4f}")
print(f"Residual MLP_SGD - recall: {results['Residual MLP_SGD']['recall']:.4f}")
print(f"Residual MLP_SGD - f1_score: {results['Residual MLP_SGD']['f1_score']:.4f}")



Training Residual MLP with Adam...
Saved Residual MLP with Adam weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Residual MLP_Adam - Accuracy: 0.9186, ROC-AUC: 0.9464
Residual MLP_Adam - precision: 0.7893
Residual MLP_Adam - recall: 0.6758
Residual MLP_Adam - f1_score: 0.7282

Training Residual MLP with RMSprop...
Saved Residual MLP with RMSProp weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
Residual MLP_RMSprop - Accuracy: 0.9146, ROC-AUC: 0.9473
Residual MLP_RMSprop - precision: 0.7533
Residual MLP_RMSprop - recall: 0.7003
Residual MLP_RMSprop - f1_score: 0.7258

Training Residual MLP with SGD...
Saved Residual MLP with SGD weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Residual MLP_SGD - Accuracy: 0.9200, ROC-AUC: 0.9494
Residual MLP_SGD - precision: 0.7477
Residual MLP_SGD - recall: 0.7615
Residual MLP_SGD - f1_score: 0.7545


In [10]:
print("\nTraining CNN-inspired Model with Adam...")
cnn_mlp_adam = build_cnn_inspired_mlp(optimizers['Adam'])
cnn_mlp_adam.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
cnn_mlp_adam.save('cnn_mlp_adam.keras')
print("Saved CNN MLP with Adam weights")
y_pred_proba = cnn_mlp_adam.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['CNN-inspired Model_Adam'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['CNN-inspired Model_Adam'] = cnn_mlp_adam
print(f"CNN-inspired Model_Adam - Accuracy: {results['CNN-inspired Model_Adam']['accuracy']:.4f}, ROC-AUC: {results['CNN-inspired Model_Adam']['roc_auc']:.4f}")
print(f"CNN-inspired Model_Adam - precision: {results['CNN-inspired Model_Adam']['precision']:.4f}")
print(f"CNN-inspired Model_Adam - recall: {results['CNN-inspired Model_Adam']['recall']:.4f}")
print(f"CNN-inspired Model_Adam - f1_score: {results['CNN-inspired Model_Adam']['f1_score']:.4f}")

print("\nTraining CNN-inspired Model with RMSprop...")
cnn_mlp_rmsprop = build_cnn_inspired_mlp(optimizers['RMSprop'])
cnn_mlp_rmsprop.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
cnn_mlp_rmsprop.save('cnn_mlp_rmsprop.keras')
print("Saved CNN MLP with RMSProp weights")
y_pred_proba = cnn_mlp_rmsprop.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['CNN-inspired Model_RMSprop'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['CNN-inspired Model_RMSprop'] = cnn_mlp_rmsprop
print(f"CNN-inspired Model_RMSprop - Accuracy: {results['CNN-inspired Model_RMSprop']['accuracy']:.4f}, ROC-AUC: {results['CNN-inspired Model_RMSprop']['roc_auc']:.4f}")
print(f"CNN-inspired Model_RMSprop - precision: {results['CNN-inspired Model_RMSprop']['precision']:.4f}")
print(f"CNN-inspired Model_RMSprop - recall: {results['CNN-inspired Model_RMSprop']['recall']:.4f}")
print(f"CNN-inspired Model_RMSprop - f1_score: {results['CNN-inspired Model_RMSprop']['f1_score']:.4f}")

print("\nTraining CNN-inspired Model with SGD...")
cnn_mlp_sgd = build_cnn_inspired_mlp(optimizers['SGD'])
cnn_mlp_sgd.fit(X_train_smote, y_train_smote, epochs=15, batch_size=32, validation_split=0.2, verbose=0, callbacks=[early_stopping], class_weight=class_weight_dict)
cnn_mlp_sgd.save('cnn_mlp_sgd.keras')
print("Saved CNN MLP with SGD weights")
y_pred_proba = cnn_mlp_sgd.predict(X_test_poly)
y_pred = (y_pred_proba > 0.5).astype("int32")
report = classification_report(y_test, y_pred, output_dict=True)
roc_auc = roc_auc_score(y_test, y_pred_proba)
results['CNN-inspired Model_SGD'] = {
    'accuracy': report['accuracy'],
    'precision': report['1']['precision'],
    'recall': report['1']['recall'],
    'f1_score': report['1']['f1-score'],
    'roc_auc': roc_auc
}
best_models['CNN-inspired Model_SGD'] = cnn_mlp_sgd
print(f"CNN-inspired Model_SGD - Accuracy: {results['CNN-inspired Model_SGD']['accuracy']:.4f}, ROC-AUC: {results['CNN-inspired Model_SGD']['roc_auc']:.4f}")
print(f"CNN-inspired Model_SGD - precision: {results['CNN-inspired Model_SGD']['precision']:.4f}")
print(f"CNN-inspired Model_SGD - recall: {results['CNN-inspired Model_SGD']['recall']:.4f}")
print(f"CNN-inspired Model_SGD - f1_score: {results['CNN-inspired Model_SGD']['f1_score']:.4f}")


Training CNN-inspired Model with Adam...
Saved CNN MLP with Adam weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
CNN-inspired Model_Adam - Accuracy: 0.9205, ROC-AUC: 0.9757
CNN-inspired Model_Adam - precision: 0.6895
CNN-inspired Model_Adam - recall: 0.9235
CNN-inspired Model_Adam - f1_score: 0.7895

Training CNN-inspired Model with RMSprop...
Saved CNN MLP with RMSProp weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
CNN-inspired Model_RMSprop - Accuracy: 0.9299, ROC-AUC: 0.9755
CNN-inspired Model_RMSprop - precision: 0.7284
CNN-inspired Model_RMSprop - recall: 0.9021
CNN-inspired Model_RMSprop - f1_score: 0.8060

Training CNN-inspired Model with SGD...
Saved CNN MLP with SGD weights
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
CNN-inspired Model_SGD - Accuracy: 0.9289, ROC-AUC: 0.9786
CNN-inspired Model_SGD - precision: 0.7237
CNN-inspired Model_SGD - recall: 0.9052
CNN-inspired Model_SGD - f1_score: 0.8043


In [14]:
for model_name, optimizers_list in models.items():
    for opt in optimizers_list:
        model_key = f"{model_name}_{opt.lower()}"

        local_model_path = f"{model_key}.keras"

        drive_model_path = os.path.join(drive_model_dir, f"{model_key}.keras")
        tf.io.gfile.copy(local_model_path, drive_model_path, overwrite=True)
        print(f"Copied {model_key} to Google Drive: {drive_model_path}")


Copied simple_mlp_adam to Google Drive: /content/drive/MyDrive/ML_Churn_Models/simple_mlp_adam.keras
Copied simple_mlp_rmsprop to Google Drive: /content/drive/MyDrive/ML_Churn_Models/simple_mlp_rmsprop.keras
Copied simple_mlp_sgd to Google Drive: /content/drive/MyDrive/ML_Churn_Models/simple_mlp_sgd.keras
Copied tabnet_adam to Google Drive: /content/drive/MyDrive/ML_Churn_Models/tabnet_adam.keras
Copied tabnet_rmsprop to Google Drive: /content/drive/MyDrive/ML_Churn_Models/tabnet_rmsprop.keras
Copied tabnet_sgd to Google Drive: /content/drive/MyDrive/ML_Churn_Models/tabnet_sgd.keras
Copied transformer_adam to Google Drive: /content/drive/MyDrive/ML_Churn_Models/transformer_adam.keras
Copied transformer_rmsprop to Google Drive: /content/drive/MyDrive/ML_Churn_Models/transformer_rmsprop.keras
Copied transformer_sgd to Google Drive: /content/drive/MyDrive/ML_Churn_Models/transformer_sgd.keras
Copied residual_mlp_adam to Google Drive: /content/drive/MyDrive/ML_Churn_Models/residual_mlp_ada

In [29]:
model_mapping = {
    'Simple MLP': 'simple_mlp',
    'TabNet-inspired Model': 'tabnet',
    'Transformer Model': 'transformer',
    'Residual MLP': 'residual_mlp',
    'CNN-inspired Model': 'cnn_mlp'
}

optimizer_mapping = {
    'Adam': 'adam',
    'RMSprop': 'rmsprop',
    'SGD': 'sgd',
}

def predict_churn(
    model_choice,
    optimizer_choice,
    Customer_Age, Gender, Dependent_count, Education_Level,
    Marital_Status, Income_Category, Card_Category,
    Months_on_book, Total_Relationship_Count, Months_Inactive_12_mon,
    Contacts_Count_12_mon, Credit_Limit, Total_Revolving_Bal,
    Avg_Open_To_Buy, Total_Amt_Chg_Q4_Q1, Total_Trans_Amt,
    Total_Trans_Ct, Total_Ct_Chg_Q4_Q1, Avg_Utilization_Ratio
):
    vals = [Customer_Age, Gender, Dependent_count, Education_Level,
            Marital_Status, Income_Category, Card_Category,
            Months_on_book, Total_Relationship_Count, Months_Inactive_12_mon,
            Contacts_Count_12_mon, Credit_Limit, Total_Revolving_Bal,
            Avg_Open_To_Buy, Total_Amt_Chg_Q4_Q1, Total_Trans_Amt,
            Total_Trans_Ct, Total_Ct_Chg_Q4_Q1, Avg_Utilization_Ratio]
    if any(v is None or (isinstance(v, str) and not v) for v in vals):
        return "Please enter all required values"

    model_key = model_mapping.get(model_choice)
    if not model_key:
        return f"Invalid model selected: {model_choice}"

    optimizer_key = optimizer_mapping.get(optimizer_choice)
    if not optimizer_key:
        return f"Invalid optimizer selected: {optimizer_choice}"

    model_filename = f"{model_key}_{optimizer_key}.keras"

    if not os.path.exists(model_filename):
        return f"Model weights not found at {model_filename}. Please train and save the model first."

    try:
        model = load_model(model_filename)
        print(f"Loaded model with weights: {model_filename}")
    except Exception as e:
        return f"Error loading model weights: {e}"

    df = pd.DataFrame([{
        'Customer_Age': Customer_Age,
        'Gender': Gender,
        'Dependent_count': Dependent_count,
        'Education_Level': Education_Level,
        'Marital_Status': Marital_Status,
        'Income_Category': Income_Category,
        'Card_Category': Card_Category,
        'Months_on_book': Months_on_book,
        'Total_Relationship_Count': Total_Relationship_Count,
        'Months_Inactive_12_mon': Months_Inactive_12_mon,
        'Contacts_Count_12_mon': Contacts_Count_12_mon,
        'Credit_Limit': Credit_Limit,
        'Total_Revolving_Bal': Total_Revolving_Bal,
        'Avg_Open_To_Buy': Avg_Open_To_Buy,
        'Total_Amt_Chg_Q4_Q1': Total_Amt_Chg_Q4_Q1,
        'Total_Trans_Amt': Total_Trans_Amt,
        'Total_Trans_Ct': Total_Trans_Ct,
        'Total_Ct_Chg_Q4_Q1': Total_Ct_Chg_Q4_Q1,
        'Avg_Utilization_Ratio': Avg_Utilization_Ratio
    }])
    df = df.rename(columns={'Total_Amt_Chg_Q4_Q1': 'Total_Amt_Chng_Q4_Q1',
                           'Total_Ct_Chg_Q4_Q1': 'Total_Ct_Chng_Q4_Q1'})

    try:
        X_proc = preprocessor.transform(df)
        X_poly = poly.transform(X_proc)

        proba = model.predict(X_poly, verbose=0)[0][0]
        return "Churn" if proba > 0.5 else "No Churn"
    except Exception as e:
        print(f"Error in predict_churn: {e}")
        return f"Error: {e}"


In [30]:
input_components = [
    gr.Dropdown(label="Model", choices=list(model_mapping.keys())),
    gr.Dropdown(label="Optimizer", choices=list(optimizer_mapping.keys())),
    gr.Number(label="Customer_Age", value=18, minimum=18, maximum=100, step=1),
    gr.Dropdown(label="Gender", choices=["M", "F"]),
    gr.Number(label="Dependent_count", minimum=0, maximum=10, step=1),
    gr.Dropdown(
        label="Education_Level",
        choices=["High School", "Graduate", "Uneducated", "Unknown",
                 "College", "Post-Graduate", "Doctorate"]
    ),
    gr.Dropdown(
        label="Marital_Status",
        choices=["Married", "Single", "Unknown", "Divorced"]
    ),
    gr.Dropdown(
        label="Income_Category",
        choices=["Less than $40K", "$40K - $60K", "$60K - $80K",
                 "$80K - $120K", "$120K +", "Unknown"]
    ),
    gr.Dropdown(
        label="Card_Category",
        choices=["Blue", "Gold", "Silver", "Platinum"]
    ),
    gr.Number(label="Months_on_book", minimum=0, maximum=100, step=1),
    gr.Number(label="Total_Relationship_Count", value = 1, minimum=1, maximum=10, step=1),
    gr.Number(label="Months_Inactive_12_mon", minimum=0, maximum=12, step=1),
    gr.Number(label="Contacts_Count_12_mon", minimum=0, maximum=10, step=1),
    gr.Number(label="Credit_Limit", minimum=0, maximum=50000, step=100),
    gr.Number(label="Total_Revolving_Bal", minimum=0, maximum=5000, step=10),
    gr.Number(label="Avg_Open_To_Buy", minimum=0, maximum=50000, step=100),
    gr.Number(label="Total_Amt_Chg_Q4_Q1", minimum=0, maximum=5, step=0.01),
    gr.Number(label="Total_Trans_Amt", minimum=0, maximum=30000, step=10),
    gr.Number(label="Total_Trans_Ct", minimum=0, maximum=200, step=1),
    gr.Number(label="Total_Ct_Chg_Q4_Q1", minimum=0, maximum=5, step=0.01),
    gr.Number(label="Avg_Utilization_Ratio", minimum=0, maximum=1, step=0.001)
]

iface = gr.Interface(
    fn=predict_churn,
    inputs=input_components,
    outputs=gr.Textbox(label="Prediction"),
    title="Customer Churn Prediction",
    description="Select a model and optimizer, then enter customer features to predict churn."
)

if __name__ == "__main__":
    iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://826035803a6d7e1dd0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
